<a href="https://colab.research.google.com/github/ananyagoyal1/Machine-Learning/blob/main/KNN_with_HOG_Features_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from fastai.vision.all import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from skimage.feature import hog
from skimage import io, color
from skimage import exposure

In [ ]:
# Load the image data
path = "/content/drive/MyDrive/EYE"
data = ImageDataLoaders.from_folder(path, train='.', valid_pct=0.2, item_tfms=Resize(224),
                                     batch_tfms=[*aug_transforms(size=224), Normalize.from_stats(*imagenet_stats)],
                                     size=224, num_workers=4)

In [ ]:
# Flatten the images to use raw pixel values as features
def flatten_images(dataloader):
    features = []
    labels = []
    for xb, yb in dataloader:
        features.append(xb.view(xb.size(0), -1).numpy())
        labels.append(yb.numpy())
    return np.concatenate(features), np.concatenate(labels)

In [ ]:
train_features, train_labels = flatten_images(data.train)
valid_features, valid_labels = flatten_images(data.valid)

In [ ]:
# Fit the StandardScaler on the training features
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)


In [ ]:
# Create a KNN classifier pipeline
knn_pipeline = Pipeline([
    ('knn', KNeighborsClassifier(n_neighbors=5))  # You can adjust the number of neighbors (n_neighbors) as needed
])


In [ ]:
# Fit the KNN classifier on the scaled training features
knn_pipeline.fit(train_features_scaled, train_labels)

Pipeline(steps=[('knn', KNeighborsClassifier())])

In [ ]:
# Scale the validation features using the same scaler
valid_features_scaled = scaler.transform(valid_features)

In [ ]:
# Use the KNN classifier to predict the labels of the validation features
predicted_labels = knn_pipeline.predict(valid_features_scaled)

In [ ]:
# Calculate the accuracy of the KNN classifier
accuracy = (predicted_labels == valid_labels).mean()
print(f"Validation accuracy: {accuracy}")

Validation accuracy: 0.9090909090909091


In [ ]:
def extract_hog_features(dataloader):
    features = []
    labels = []
    for xb, yb in dataloader:
        for i in range(xb.size(0)):
            # Transpose the image array to have shape (height, width, 3)
            image = xb[i].numpy().transpose(1, 2, 0)
            gray_image = color.rgb2gray(image)
            hog_features = hog(gray_image, visualize=False)
            features.append(hog_features)
        labels.append(yb.numpy())
    return np.array(features), np.concatenate(labels)

In [ ]:
train_features1, train_labels1 = extract_hog_features(data.train)
valid_features1, valid_labels1 = extract_hog_features(data.valid)

In [ ]:
# Fit the StandardScaler on the training features
scaler = StandardScaler()
train_features_scaled1 = scaler.fit_transform(train_features1)

In [ ]:
# Fit the KNN classifier on the scaled training features
knn_pipeline.fit(train_features_scaled1, train_labels1)

Pipeline(steps=[('knn', KNeighborsClassifier())])

In [ ]:
# Scale the validation features using the same scaler
valid_features_scaled1 = scaler.transform(valid_features1)

In [ ]:
# Use the KNN classifier to predict the labels of the validation features
predicted_labels = knn_pipeline.predict(valid_features_scaled1)

In [ ]:
# Calculate the accuracy of the KNN classifier
accuracy = (predicted_labels == valid_labels).mean()
print(f"Validation accuracy: {accuracy}")

Validation accuracy: 0.9393939393939394
